In [1]:
from transformers import BertTokenizerFast, BertForQuestionAnswering
import torch
from torch.utils.data import DataLoader, TensorDataset
from DataGeneration import DataGeneration  # Importing the DataGeneration package

/home/marc/miniconda3/envs/ArchiveReader/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)
# !python -m spacy download en_core_web_lg

2.3.0+cu121


In [3]:
# Load tokenizer and pre-trained BERT model
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

# Instantiate DataGeneration class
data_generator = DataGeneration()

# Generate data
raw_data_file = "raw_datasets\SUAS-Competition-FALL2023-Final-Report.txt"
raw_data_file_linux = "raw_datasets/SUAS-Competition-FALL2023-Final-Report.txt"
train_contexts, train_questions, train_answers = data_generator.generate_data(raw_data_file_linux)

/home/marc/miniconda3/envs/ArchiveReader/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
[nltk_data] Downloading package punkt to /home/marc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/marc/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/marc/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
print(train_contexts)

['suas competition software team fall 2023 final report authors marc cruz abdul kalam syed max gross joshua estrada jason mar josh ng ethan tarrer sarkis gafayan rubayet mujahid david jackson status done date sep 21 2023 relative links suas competition software team overview suas competition technical design document suas competition machine learning models 1. introduction two 2. fall 2023 progress three 2.1 timeline three 2.2 recruitment august september four 2.3 learning phase august october four 2.3.1 odlc four 2.3.2 obstacle avoidance five 2.4 odlc design five 2.5 hardware trade studies seven 2.5.1 odlc trade studies seven 2.5.1.1 obc trade study seven 2.5.1.1 camera trade study eight 2.5.2 obstacle avoidance trade study nine 2.6 data collection dataset phase october december ten 2.6.1 standardized object shape ml model ten 2.6.2 standardized object shape alphanumeric color ml model twelve 2.6.3 standardized object alphanumeric ml model thirteen 2.7 modeling phase october february 

In [5]:
print(train_questions)

[['What is 2023?', 'Can you provide details about 2023?', 'Explain the significance of 2023.', 'What are the properties of 2023?'], ['What is yearly?', 'Can you provide details about yearly?', 'Explain the significance of yearly.', 'What are the properties of yearly?'], ['What is semicircle?', 'Can you provide details about semicircle?', 'Explain the significance of semicircle.', 'What are the properties of semicircle?'], ['What is pentagon?', 'Can you provide details about pentagon?', 'Explain the significance of pentagon.', 'What are the properties of pentagon?'], ['What is year?', 'Can you provide details about year?', 'Explain the significance of year.', 'What are the properties of year?'], ['What is mader?', 'Can you provide details about mader?', 'Explain the significance of mader.', 'What are the properties of mader?'], ['What is v8?', 'Can you provide details about v8?', 'Explain the significance of v8.', 'What are the properties of v8?'], ['What is winter?', 'Can you provide d

In [6]:
print(train_answers)

[[{'start_index': 36, 'end_index': 40}, {'start_index': 36, 'end_index': 40}, {'start_index': 36, 'end_index': 40}, {'start_index': 36, 'end_index': 40}, {'start_index': 36, 'end_index': 40}], [{'start_index': 37, 'end_index': 43}, {'start_index': 37, 'end_index': 43}, {'start_index': 37, 'end_index': 43}, {'start_index': 37, 'end_index': 43}, {'start_index': 37, 'end_index': 43}], [{'start_index': 153, 'end_index': 163}, {'start_index': 153, 'end_index': 163}, {'start_index': 153, 'end_index': 163}, {'start_index': 153, 'end_index': 163}, {'start_index': 153, 'end_index': 163}], [{'start_index': 198, 'end_index': 206}, {'start_index': 198, 'end_index': 206}, {'start_index': 198, 'end_index': 206}, {'start_index': 198, 'end_index': 206}, {'start_index': 198, 'end_index': 206}], [{'start_index': 37, 'end_index': 41}, {'start_index': 37, 'end_index': 41}, {'start_index': 37, 'end_index': 41}, {'start_index': 37, 'end_index': 41}, {'start_index': 37, 'end_index': 41}], [{'start_index': 54

In [35]:
backup_train_contexts = train_contexts[:]
backup_train_questions = train_questions[:]
backup_train_answers = train_answers[:]

In [38]:
train_contexts = backup_train_contexts[:]
train_questions = backup_train_questions[:]
train_answers = backup_train_answers[:]

In [37]:
# Restore the original data
train_contexts = backup_train_contexts[:]
train_questions = backup_train_questions[:]
train_answers = backup_train_answers[:]

# Convert contexts and questions to strings
train_contexts = [str(context) for context in train_contexts]
train_questions = [str(question) for question in train_questions]

# Tokenize inputs
inputs = tokenizer(train_contexts, 
                   train_questions, 
                   padding=True, 
                   truncation=True, 
                   return_tensors='pt',
                   max_length=512, 
                   add_special_tokens=True)

# Convert answers to answer indices
start_positions = []
end_positions = []
for answers in train_answers:
    entity_start_positions = []
    entity_end_positions = []
    for ans in answers:
        start_char = ans['start_index']
        end_char = ans['end_index']
        # Find corresponding token indices using offsets
        start_token = inputs.char_to_token(0, start_char)
        end_token = inputs.char_to_token(0, end_char)
        entity_start_positions.append(start_token)
        entity_end_positions.append(end_token)
    start_positions.append(entity_start_positions)
    end_positions.append(entity_end_positions)

# Filter out None values from start_positions and end_positions
start_positions = [[pos for pos in sublist if pos is not None] for sublist in start_positions]
end_positions = [[pos for pos in sublist if pos is not None] for sublist in end_positions]

# Filter out samples where end positions are empty
non_empty_end_positions_indices = [i for i, pos in enumerate(end_positions) if len(pos) > 0]

# Use the filtered indices to select corresponding tensors
filtered_inputs = {key: value[non_empty_end_positions_indices] for key, value in inputs.items()}
filtered_start_positions_tensor = [start_positions_tensor[i] for i in non_empty_end_positions_indices]
filtered_end_positions_tensor = [end_positions_tensor[i] for i in non_empty_end_positions_indices]

# Check the lengths of the tensors
num_samples = len(filtered_inputs['input_ids'])
assert len(filtered_inputs['token_type_ids']) == num_samples, "Size mismatch between token_type_ids and input_ids"
assert len(filtered_inputs['attention_mask']) == num_samples, "Size mismatch between attention_mask and input_ids"
assert len(filtered_start_positions_tensor) == num_samples, "Size mismatch between start_positions and input_ids"
assert len(filtered_end_positions_tensor) == num_samples, "Size mismatch between end_positions and input_ids"

print(len(filtered_inputs['input_ids']))
print(len(filtered_inputs['token_type_ids']))
print(len(filtered_inputs['attention_mask']))
print(len(filtered_start_positions_tensor))
print(len(filtered_end_positions_tensor))

print(filtered_inputs['input_ids'].shape)
print(filtered_inputs['token_type_ids'].shape)
print(filtered_inputs['attention_mask'].shape)
print(filtered_start_positions_tensor[0].shape)  # Print shape of the first tensor in the list
print(filtered_end_positions_tensor[0].shape)    # Print shape of the first tensor in the list

# Create a PyTorch dataset
train_dataset = TensorDataset(filtered_inputs['input_ids'], filtered_inputs['token_type_ids'], filtered_inputs['attention_mask'],
                              *filtered_start_positions_tensor, *filtered_end_positions_tensor)

# Define your DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, token_type_ids, attention_mask, start_positions, end_positions = batch
        outputs = model(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask,
                        start_positions=start_positions, end_positions=end_positions)
        loss = loss_fn(outputs.start_logits, start_positions.view(-1)) + loss_fn(outputs.end_logits, end_positions.view(-1))
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {total_loss}')

# Save the trained model
model.save_pretrained('bert_qa_model')


12
12
12
12
12
torch.Size([12, 512])
torch.Size([12, 512])
torch.Size([12, 512])
torch.Size([5])
torch.Size([5])


AssertionError: Size mismatch between tensors